In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2

# Load YOLOv8 model (you can switch to yolov8s.pt or your custom model)
model = YOLO('yolov8n.pt')
CAR_CLASS_ID = 2  # COCO class ID for 'car'

# Function to detect and annotate cars in a frame
def detect_cars(frame):
    results = model(frame)[0]
    for box in results.boxes:
        cls = int(box.cls[0])
        if cls == CAR_CLASS_ID:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            label = f"Car {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

# Process selected video
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = detect_cars(frame)
        cv2.imshow("YOLOv8 Car Detection", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to stop
            break

    cap.release()
    cv2.destroyAllWindows()

# GUI: Browse and select a video file
def select_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")])
    if file_path:
        process_video(file_path)

# Set up GUI window
root = tk.Tk()
root.title("YOLOv8 Car Detector")
root.geometry("350x180")
root.configure(bg="#f0f0f0")

# GUI Elements
title = tk.Label(root, text="YOLOv8 Car Detection", font=("Helvetica", 16, "bold"), bg="#f0f0f0")
title.pack(pady=10)

instruction = tk.Label(root, text="Click the button to select a video", font=("Helvetica", 12), bg="#f0f0f0")
instruction.pack(pady=5)

browse_button = tk.Button(root, text="Browse Video", command=select_video, font=("Helvetica", 12),
                          bg="#4CAF50", fg="white", padx=10, pady=5)
browse_button.pack(pady=15)

# Start GUI
root.mainloop()
